# Import libraries

In [1]:
import re
import pandas as pd
import pdfplumber
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span
nlp=spacy.load('en_core_web_sm')
import joblib

# Data Segmentation by process

In [2]:
def process_segmentation() :
    ch=''
    names =''
    start = 0
    process_df = pd.DataFrame(columns=['Process Name','start', 'Corpus'])
    with pdfplumber.open("docs/PMBOK 5th.pdf") as pdf:
        for i in range(338,380) :
            page = (pdf.pages[i]).extract_text()
            for line in page.split('\n') :
                if re.match('\d{2}.\d\s[A-Za-z]+\s[A-Za-z]',line) :
                    if start != 0 :
                        process_df = process_df.append({'Process Name': name,'start':start, 'Corpus':ch[1:]}, ignore_index=True)

                    ch = ''
                    start = i
                    name = line[5:]
                ch=ch+'. '+line

    process_df = process_df.append({'Process Name': name,'start':start, 'Corpus':ch[1:]}, ignore_index=True)

    return (process_df)

## Creating our df DataFrame that contains The Process name, starting page and the corpus

In [3]:
df = process_segmentation()

In [4]:
process1 = df['Corpus'][0]
process2 = df['Corpus'][1]
process3 = df['Corpus'][2]
process4 = df['Corpus'][3]
process5 = df['Corpus'][4]
process6 = df['Corpus'][5]

In [5]:
df

,Process Name,start,Corpus
0,Plan risk Management,338,11.1 Plan risk Management. Plan Risk Manageme...
1,Identify risks,344,11.2 Identify risks. Identify Risks is the pr...
2,Perform Qualitative risk Analysis,353,11.3 Perform Qualitative risk Analysis. Perfo...
3,Perform Quantitative risk Analysis,358,11.4 Perform Quantitative risk Analysis. Perf...
4,Plan risk responses,367,11.5 Plan risk responses. Plan Risk Responses...
5,control risks,374,11.6 control risks. Control Risks is the proc...


## Here we will merge the hole corpus 

In [6]:
corpus = process1 + process2 + process3 + process4 + process5 + process6

## We will be using regex to locate the : <br>
  <li>Input titles</li>
  <li>tools_and_techniques</li>
  <li>outputs_titles</li>
  <li>get_subtiltles</li>
  <li>title</li>
  

    

In [7]:
inputs_titles = re.compile(r'\d+\..+: Inputs\n')
tools_and_techniques = re.compile(r'\d+\..+: tools and techniques\n')
outputs_titles = re.compile(r'\d+\..+: outputs\n')
get_subtiltles = re.compile(r'[^|\n](\d+\.){2,4}\d+ [a-z|\s|A-Z]+(\d+)?\n')
title = re.compile(r'\d+ (.+): Inputs\n')

## Now let's get the text and clean it

In [8]:
ch=''
with pdfplumber.open("docs/PMBOK 5th.pdf") as temp:
    for i in range(339,380) :
        first_page = temp.pages[i]
        ch=ch+first_page.extract_text()

In [9]:
def clean(text):
    # Remove numbers
    text = re.sub('[•-]','',text)
    text = re.sub("\s\s+" , " ", text)
    return text 

In [10]:
ch = ch.replace('11 - PROJECT RISK MANAGEMENT\n','')
ch = clean(ch)

## Our text is now clean and can be segmented

In [11]:
sections = inputs_titles.finditer(ch)
df = []
inputs = []
tt = []
outputss=[]
lst_sec = [elem for elem in sections]
for i in range(0,len(lst_sec)):
    if i == (len(lst_sec)-1):
        print(lst_sec[i].group())
        print("processus : ", title.search(lst_sec[i].group()).group(1))
        df.append(title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        str_between = ch[int(lst_sec[i].span()[1]):]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtiltles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
#####End liste des inputs
#[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtiltles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"
        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
######End liste des tt
#[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtiltles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
######End liste des outputs
#[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro

    else:
        print(lst_sec[i].group())
        # print("processus : ", title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        df.append(title.search(lst_sec[i].group()).group(1))
        str_between = ch[int(lst_sec[i].span()[1]):int(lst_sec[i+1].span()[0])]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtiltles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)

        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
    #####End liste des inputs
    #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtiltles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"

        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
    ######End liste des tt
    #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtiltles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
        ######End liste des outputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro
        print('-'*70)

11.1.1 Plan risk Management: Inputs

	11.1.1.1 Project Management Plan
	11.1.1.2 Project charter
	11.1.1.3 Stakeholder register
	11.1.1.4 Enterprise Environmental Factors
	11.1.1.5 organizational Process Assets
11.1.2 Plan risk Management: tools and techniques

	11.1.2.1 Analytical techniques
	11.1.2.2 Expert Judgment
	.11.1.2.3 Meetings
11.1.3 Plan risk Management: outputs

	11.1.3.1 risk Management Plan
----------------------------------------------------------------------
11.2.1 Identify risks: Inputs

	11.2.1.1 risk Management Plan 11
	11.2.1.2 cost Management Plan
	11.2.1.3 Schedule Management Plan
	11.2.1.4 Quality Management Plan
	.11.2.1.5 Human resource Management Plan
	11.2.1.6 Scope Baseline
	11.2.1.7 Activity cost Estimates
	11.2.1.8 Activity duration Estimates
	11.2.1.9 Stakeholder register
	.11.2.1.10 Project documents
	11.2.1.11 Procurement documents
	11.2.1.12 Enterprise Environmental Factors
	.11.2.1.13 organizational Process Assets
11.2.2 Identify risks: tools and tec

## Since we have many objects we will be creating for each one a variable dynamically usin exec 

## Every Process has inputs, tools&technoques and outputs. <br>
So every process will have this structure: <br>
For example for process1 the variables will be created like this <br>
    <li>To access Process <b>1</b>'s <b>inputs</b> the variable name will be : <b>p1_inputs</b> </li>
    <li>To access Process <b>1</b>'s <b>tools and techniques</b> the variable name will be : <b>p1_tt</b></li>
    <li>To access Process <b>1</b>'s <b>outputs</b> the variable name will be : <b>p1_outputs</b></li>

In [12]:
for i in inputs:
    a=i[0]
    del(i[0])

    exec('%s = %s' %(a, i[:]))

In [13]:
for i in outputss:
    a=i[0]
    del(i[0])

    exec('%s = %s' %(a, i[:]))

In [14]:
for i in tt:
    a=i[0]
    del(i[0])

    exec('%s = %s' %(a, i[:]))

## We will repeat the process to locate the annotation of every input tools and techniques and the output of every process

So every process will have this structure: <br>
For example for process1 the variables will be created like this <br>
    <li>To access <b>The Input annotation</b> of Process <b>1</b> the variable name will be : <b>in_p1</b> </li>
    <li>To access <b>The Tools and Techniques annotation</b> of Process <b>1</b> the variable name will be : <b>tt_p1</b></li>
    <li>To access <b>The Output annotation</b> of Process <b>1</b> the variable name will be : <b>out_p1</b></li>

In [15]:
for i in range(1,7):
    exec(f'tt_p{i} = re.search(\'\d+\..+tools and techniques(.*)\d+\..+outputs\', process{i}).group(1)')
    exec(f'in_p{i} = re.search(\'\d+\..+Inputs(.*)\d+\..+tools and techniques\', process{i}).group(1)')
    exec(f'out_p{i} = re.search(\'\d+\..+outputs(.*)\', process{i}).group(1)')

In [16]:
def make_annotation(element, text, process, iot):
    ann = []
    for i in range(1,len(element)):
        ann.append(re.search('11.'+str(process)+'.'+str(iot)+'.'+str(i)+'(.*)11.'+str(process)+'.'+str(iot)+'.'+str(i+1)+'', text).group(1))
    ann.append(re.search('11.'+str(process)+'.'+str(iot)+'.'+str(element.index(element[-1])+1)+'(.*)', text).group(1))
    return ann

### Inputs

In [17]:
an_in_p1 = make_annotation(p1_inputs, in_p1, 1, 1)
an_in_p2 = make_annotation(p2_inputs, in_p2, 2, 1)
an_in_p3 = make_annotation(p3_inputs, in_p3, 3, 1)
an_in_p4 = make_annotation(p4_inputs, in_p4, 4, 1)
an_in_p5 = make_annotation(p5_inputs, in_p5, 5, 1)
an_in_p6 = make_annotation(p6_inputs, in_p6, 6, 1)

### Tools&Techniques

In [18]:
an_tt_p1 = make_annotation(p1_tt, tt_p1, 1, 2)
an_tt_p2 = make_annotation(p2_tt, tt_p2, 2, 2)
an_tt_p3 = make_annotation(p3_tt, tt_p3, 3, 2)
an_tt_p4 = make_annotation(p4_tt, tt_p4, 4, 2)
an_tt_p5 = make_annotation(p5_tt, tt_p5, 5, 2)
an_tt_p6 = make_annotation(p6_tt, tt_p6, 6, 2)

### Outputs

In [19]:
an_out_p1 = make_annotation(p1_outputs, out_p1, 1, 3)
an_out_p2 = make_annotation(p2_outputs, out_p2, 2, 3)
an_out_p3 = make_annotation(p3_outputs, out_p3, 3, 3)
an_out_p4 = make_annotation(p4_outputs, out_p4, 4, 3)
an_out_p5 = make_annotation(p5_outputs, out_p5, 5, 3)
an_out_p6 = make_annotation(p6_outputs, out_p6, 6, 3)

## We noticed that there are annotations that are described with figures so we cleaned the output

In [20]:
def clean_input(text):
    a = []
    for t in text:
        if re.findall('in Section(.*)', t):
            t = re.sub('[\d]+','',t)
            t = re.sub('\s[.]+','',t)
            t = re.search('in Section(.*)', t).group(1)
        a.append(t)
    return a

In [21]:
an_in_p1 = clean_input(an_in_p1)
an_in_p2 = clean_input(an_in_p2)
an_in_p3 = clean_input(an_in_p3)
an_in_p4 = clean_input(an_in_p4)
an_in_p5 = clean_input(an_in_p5)
an_in_p6 = clean_input(an_in_p6)

## To prepare our data to the ontology building, we devided our Processes into 3 variables. <br>
<li>individuals : The Title of the process elements</li>
<li>annotation: The description of the individual</li>
<li>process: The process name</li>
<li>concept: concept ['Inputs', 'Tools and techniques', 'Outputs']</li>
<li>pattern: Object + Subject + Link</li>

### Process 1

In [22]:
df_p1_in = pd.DataFrame({
    'individuals':p1_inputs, 'annotation':an_in_p1
})
df_p1_in['process'] = 'Plan Risk Management'
df_p1_in['concept'] = 'Plan Risk Management Inputs'
df_p1_in['pattern'] = '[]'

df_p1_out = pd.DataFrame({
    'individuals':p1_outputs, 'annotation':an_out_p1
})

df_p1_out['process'] = 'Plan Risk Management'
df_p1_out['concept'] = 'Plan Risk Management Outputs'
df_p1_out['pattern'] = '[]'

df_p1_tt = pd.DataFrame({
    'individuals':p1_tt, 'annotation':an_tt_p1
})

df_p1_tt['process'] = 'Plan Risk Management'
df_p1_tt['concept'] = 'Plan Risk Management TT'
df_p1_tt['pattern'] = '[]'


In [23]:
df_p1_in

,individuals,annotation,process,concept,pattern
0,Project Management Plan,Project Management Plan. In planning risk man...,Plan Risk Management,Plan Risk Management Inputs,[]
1,Project charter,The project charter can provide various input...,Plan Risk Management,Plan Risk Management Inputs,[]
2,Stakeholder register,"The stakeholder register, which contains all ...",Plan Risk Management,Plan Risk Management Inputs,[]
3,Enterprise Environmental Factors,The enterprise environmental factors that can...,Plan Risk Management,Plan Risk Management Inputs,[]
4,organizational Process Assets,The organizational process assets that can in...,Plan Risk Management,Plan Risk Management Inputs,[]


### Process 2

In [24]:
df_p2_in = pd.DataFrame({
    'individuals':p2_inputs, 'annotation':an_in_p2
})
df_p2_in['process'] = 'Identify Risks'
df_p2_in['concept'] = 'Identify Risks Inputs'
df_p2_in['pattern'] = '[]'

df_p2_out = pd.DataFrame({
    'individuals':p2_outputs, 'annotation':an_out_p2
})
df_p2_out['process'] = 'Identify Risks'
df_p2_out['concept'] = 'Identify Risks Inputs'
df_p2_out['pattern'] = '[]'

df_p2_tt = pd.DataFrame({
    'individuals':p2_tt, 'annotation':an_tt_p2
})
df_p2_tt['process'] = 'Identify Risks'
df_p2_tt['concept'] = 'Plan Risk Management TT'
df_p2_tt['pattern'] = '[]'


### Process 3

In [25]:
df_p3_in = pd.DataFrame({
    'individuals':p3_inputs, 'annotation':an_in_p3
})
df_p3_in['process'] = 'Perform Qualitative Risk Analysis'

df_p3_in['concept'] = 'Perform Qualitative Risk Analysis Inputs'
df_p3_in['pattern'] = '[]'

df_p3_out = pd.DataFrame({
    'individuals':p3_outputs, 'annotation':an_out_p3
})
df_p3_out['process'] = 'Perform Qualitative Risk Analysis'

df_p3_out['concept'] = 'Perform Qualitative Risk Analysis Outputs'
df_p3_out['pattern'] = '[]'

df_p3_tt = pd.DataFrame({
    'individuals':p3_tt, 'annotation':an_tt_p3
})
df_p3_tt['process'] = 'Perform Qualitative Risk Analysis'

df_p3_tt['concept'] = 'Perform Qualitative Risk Analysis TT'
df_p3_tt['pattern'] = '[]'

### Process 4

In [26]:
df_p4_in = pd.DataFrame({
    'individuals':p4_inputs, 'annotation':an_in_p4
})
df_p4_in['process'] = 'Perform Quantitative Risk Analysis'

df_p4_in['concept'] = 'Perform Quantitative Risk Analysis Inputs'
df_p4_in['pattern'] = '[]'

df_p4_out = pd.DataFrame({
    'individuals':p4_outputs, 'annotation':an_out_p4
})
df_p4_out['process'] = 'Perform Quantitative Risk Analysis'

df_p4_out['concept'] = 'Perform Quantitative Risk Analysis Outputs'
df_p4_out['pattern'] = '[]'

df_p4_tt = pd.DataFrame({
    'individuals':p4_tt, 'annotation':an_tt_p4
})
df_p4_tt['process'] = 'Perform Quantitative Risk Analysis'

df_p4_tt['concept'] = 'Perform Quantitative Risk Analysis TT'
df_p4_tt['pattern'] = '[]'


### Process 5

In [27]:
df_p5_in = pd.DataFrame({
    'individuals':p5_inputs, 'annotation':an_in_p5
})
df_p5_in['process'] = 'Plan Risk Responses'

df_p5_in['concept'] = 'Plan Risk Responses Inputs'
df_p5_in['pattern'] = '[]'

df_p5_out = pd.DataFrame({
    'individuals':p5_outputs, 'annotation':an_out_p5
})
df_p5_out['process'] = 'Plan Risk Responses'

df_p5_out['concept'] = 'Plan Risk Responses Outputs'
df_p5_out['pattern'] = '[]'

df_p5_tt = pd.DataFrame({
    'individuals':p5_tt, 'annotation':an_tt_p5
})
df_p5_tt['process'] = 'Plan Risk Responses'

df_p5_tt['concept'] = 'Plan Risk Responses TT'
df_p5_tt['pattern'] = '[]'


### Process 6

In [28]:
df_p6_in = pd.DataFrame({
    'individuals':p6_inputs, 'annotation':an_in_p6
})

df_p6_in['concept'] = 'Control Risks Inputs'
df_p6_in['pattern'] = '[]'
df_p6_in['process'] = 'Control Risks'

df_p6_out = pd.DataFrame({
    'individuals':p6_outputs, 'annotation':an_out_p6
})

df_p6_out['concept'] = 'Control Risks Outputs'
df_p6_out['pattern'] = '[]'
df_p6_out['process'] = 'Control Risks'

df_p6_tt = pd.DataFrame({
    'individuals':p6_tt, 'annotation':an_tt_p6
})
df_p6_tt['process'] = 'Control Risks'

df_p6_tt['concept'] = 'Control Risks TT'
df_p6_tt['pattern'] = '[]'

## Now Moving to the enities and relation extraction

### Extracting Entities

In [29]:
def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""  # dependency tag of previous token in the sentence
    prv_tok_text = ""  # previous token in the sentence

    prefix = ""
    modifier = ""

    #############################################################

    for tok in nlp(sent):
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
            # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text

            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text

            ## chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier + " " + prefix + " " + tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""

            ## chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier + " " + prefix + " " + tok.text

            ## chunk 5
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
    #############################################################

    return [ent1.strip(), ent2.strip()]

In [30]:
def make_entity(df):
    entity_pairs = []
    for i in df:
        entity_pairs.append(get_entities(i))
    return entity_pairs

### Now we will create the entities

In [31]:
entity_pairs_in_1 = make_entity(df_p1_in['annotation'])
entity_pairs_in_2 = make_entity(df_p2_in['annotation'])
entity_pairs_in_3 = make_entity(df_p3_in['annotation'])
entity_pairs_in_4 = make_entity(df_p4_in['annotation'])
entity_pairs_in_5 = make_entity(df_p5_in['annotation'])
entity_pairs_in_6 = make_entity(df_p6_in['annotation'])

In [32]:
entity_pairs_tt_1 = make_entity(df_p1_tt['annotation'])
entity_pairs_tt_2 = make_entity(df_p2_tt['annotation'])
entity_pairs_tt_3 = make_entity(df_p3_tt['annotation'])
entity_pairs_tt_4 = make_entity(df_p4_tt['annotation'])
entity_pairs_tt_5 = make_entity(df_p5_tt['annotation'])
entity_pairs_tt_6 = make_entity(df_p6_tt['annotation'])

In [33]:
entity_pairs_out_1 = make_entity(df_p1_out['annotation'])
entity_pairs_out_2 = make_entity(df_p2_out['annotation'])
entity_pairs_out_3 = make_entity(df_p3_out['annotation'])
entity_pairs_out_4 = make_entity(df_p4_out['annotation'])
entity_pairs_out_5 = make_entity(df_p5_out['annotation'])
entity_pairs_out_6 = make_entity(df_p6_out['annotation'])

### During this step we will extract the relations between the words

In [34]:
def get_relation(sent):

    doc = nlp(sent)

    # Matcher class object
    matcher = Matcher(nlp.vocab)

    #define the pattern
    pattern = [ {'DEP':'ROOT'}, 
                {'DEP':'prep','OP':"?"},
                {'DEP':'agent','OP':"?"},  
                {'POS':'ADJ','OP':"?"} ] 

    matcher.add("matching_1", None, pattern)

    matches = matcher(doc)
    k = len(matches) - 1

    span = doc[matches[k][1]:matches[k][2]]

    return (span.text)

In [35]:
relations1_in = [get_relation(i) for i in df_p1_in['annotation']]
relations2_in = [get_relation(i) for i in df_p2_in['annotation']]
relations3_in = [get_relation(i) for i in df_p3_in['annotation']]
relations4_in = [get_relation(i) for i in df_p4_in['annotation']]
relations5_in = [get_relation(i) for i in df_p5_in['annotation']]
relations6_in = [get_relation(i) for i in df_p6_in['annotation']]

In [36]:
relations1_tt = [get_relation(i) for i in df_p1_tt['annotation']]
relations2_tt = [get_relation(i) for i in df_p2_tt['annotation']]
relations3_tt = [get_relation(i) for i in df_p3_tt['annotation']]
relations4_tt = [get_relation(i) for i in df_p4_tt['annotation']]
relations5_tt = [get_relation(i) for i in df_p5_tt['annotation']]
relations6_tt = [get_relation(i) for i in df_p6_tt['annotation']]

In [37]:
relations1_out = [get_relation(i) for i in df_p1_out['annotation']]
relations2_out = [get_relation(i) for i in df_p2_out['annotation']]
relations3_out = [get_relation(i) for i in df_p3_out['annotation']]
relations4_out = [get_relation(i) for i in df_p4_out['annotation']]
relations5_out = [get_relation(i) for i in df_p5_out['annotation']]
relations6_out = [get_relation(i) for i in df_p6_out['annotation']]

### Here we will groupe every thing in DataFrames

#### Inputs relations

In [38]:
subject1_in = [i[0] for i in entity_pairs_in_1]
subject2_in = [i[0] for i in entity_pairs_in_2]
subject3_in = [i[0] for i in entity_pairs_in_3]
subject4_in = [i[0] for i in entity_pairs_in_4]
subject5_in = [i[0] for i in entity_pairs_in_5]
subject6_in = [i[0] for i in entity_pairs_in_6]

target1_in = [i[1] for i in entity_pairs_in_1]
target2_in = [i[1] for i in entity_pairs_in_2]
target3_in = [i[1] for i in entity_pairs_in_3]
target4_in = [i[1] for i in entity_pairs_in_4]
target5_in = [i[1] for i in entity_pairs_in_5]
target6_in = [i[1] for i in entity_pairs_in_6]

df1_in = pd.DataFrame({'subject':subject1_in, 'property':relations1_in, 'object':target1_in })
df2_in = pd.DataFrame({'subject':subject2_in, 'property':relations2_in, 'object':target2_in })
df3_in = pd.DataFrame({'subject':subject3_in, 'property':relations3_in, 'object':target3_in })
df4_in = pd.DataFrame({'subject':subject4_in, 'property':relations4_in, 'object':target4_in })
df5_in = pd.DataFrame({'subject':subject5_in, 'property':relations5_in, 'object':target5_in })
df6_in = pd.DataFrame({'subject':subject6_in, 'property':relations6_in, 'object':target6_in })

#### Tools and Techniques relations

In [39]:
subject1_tt = [i[0] for i in entity_pairs_tt_1]
subject2_tt = [i[0] for i in entity_pairs_tt_2]
subject3_tt = [i[0] for i in entity_pairs_tt_3]
subject4_tt = [i[0] for i in entity_pairs_tt_4]
subject5_tt = [i[0] for i in entity_pairs_tt_5]
subject6_tt = [i[0] for i in entity_pairs_tt_6]

target1_tt = [i[1] for i in entity_pairs_tt_1]
target2_tt = [i[1] for i in entity_pairs_tt_2]
target3_tt = [i[1] for i in entity_pairs_tt_3]
target4_tt = [i[1] for i in entity_pairs_tt_4]
target5_tt = [i[1] for i in entity_pairs_tt_5]
target6_tt = [i[1] for i in entity_pairs_tt_6]

df1_tt = pd.DataFrame({'subject':subject1_tt, 'property':relations1_tt, 'object':target1_tt })
df2_tt = pd.DataFrame({'subject':subject2_tt, 'property':relations2_tt, 'object':target2_tt })
df3_tt = pd.DataFrame({'subject':subject3_tt, 'property':relations3_tt, 'object':target3_tt })
df4_tt = pd.DataFrame({'subject':subject4_tt, 'property':relations4_tt, 'object':target4_tt })
df5_tt = pd.DataFrame({'subject':subject5_tt, 'property':relations5_tt, 'object':target5_tt })
df6_tt = pd.DataFrame({'subject':subject6_tt, 'property':relations6_tt, 'object':target6_tt })

#### Output relation

In [40]:
subject1_out = [i[0] for i in entity_pairs_out_1]
subject2_out = [i[0] for i in entity_pairs_out_2]
subject3_out = [i[0] for i in entity_pairs_out_3]
subject4_out = [i[0] for i in entity_pairs_out_4]
subject5_out = [i[0] for i in entity_pairs_out_5]
subject6_out = [i[0] for i in entity_pairs_out_6]

target1_out = [i[1] for i in entity_pairs_out_1]
target2_out = [i[1] for i in entity_pairs_out_2]
target3_out = [i[1] for i in entity_pairs_out_3]
target4_out = [i[1] for i in entity_pairs_out_4]
target5_out = [i[1] for i in entity_pairs_out_5]
target6_out = [i[1] for i in entity_pairs_out_6]

df1_out = pd.DataFrame({'subject':subject1_out, 'property':relations1_out, 'object':target1_out })
df2_out = pd.DataFrame({'subject':subject2_out, 'property':relations2_out, 'object':target2_out })
df3_out = pd.DataFrame({'subject':subject3_out, 'property':relations3_out, 'object':target3_out })
df4_out = pd.DataFrame({'subject':subject4_out, 'property':relations4_out, 'object':target4_out })
df5_out = pd.DataFrame({'subject':subject5_out, 'property':relations5_out, 'object':target5_out })
df6_out = pd.DataFrame({'subject':subject6_out, 'property':relations6_out, 'object':target6_out })

### Now we will merge all of the DataFrames in a Big One

In [41]:
df_p1_in = pd.concat([df_p1_in, df1_in], axis=1)
df_p2_in = pd.concat([df_p2_in, df2_in], axis=1)
df_p3_in = pd.concat([df_p3_in, df3_in], axis=1)
df_p4_in = pd.concat([df_p4_in, df4_in], axis=1)
df_p5_in = pd.concat([df_p5_in, df5_in], axis=1)
df_p6_in = pd.concat([df_p6_in, df6_in], axis=1)

In [42]:
df_p1_tt = pd.concat([df_p1_tt, df1_tt], axis=1)
df_p2_tt = pd.concat([df_p2_tt, df2_tt], axis=1)
df_p3_tt = pd.concat([df_p3_tt, df3_tt], axis=1)
df_p4_tt = pd.concat([df_p4_tt, df4_tt], axis=1)
df_p5_tt = pd.concat([df_p5_tt, df5_tt], axis=1)
df_p6_tt = pd.concat([df_p6_tt, df6_tt], axis=1)

In [43]:
df_p1_out = pd.concat([df_p1_out, df1_out], axis=1)
df_p2_out = pd.concat([df_p2_out, df2_out], axis=1)
df_p3_out = pd.concat([df_p3_out, df3_out], axis=1)
df_p4_out = pd.concat([df_p4_out, df4_out], axis=1)
df_p5_out = pd.concat([df_p5_out, df5_out], axis=1)
df_p6_out = pd.concat([df_p6_out, df6_out], axis=1)

In [44]:
frames = [df_p1_in,df_p2_in,df_p3_in,df_p4_in,df_p5_in,df_p6_in,df_p1_tt,df_p2_tt,df_p3_tt,df_p4_tt,df_p5_tt,df_p6_tt,
         df_p1_out,df_p2_out,df_p3_out,df_p4_out,df_p5_out,df_p6_out]

df_concept = pd.concat(frames)

In [45]:
df_concept = df_concept.reset_index()
df_concept = df_concept.drop('index',axis=1)
df_concept = df_concept.drop('pattern',axis=1)

In [46]:
df_concept['pattern'] = df_concept['subject']+' '+df_concept['property']+' '+df_concept['object']

In [47]:
df_concept

,individuals,annotation,process,concept,subject,property,object,pattern
0,Project Management Plan,Project Management Plan. In planning risk man...,Plan Risk Management,Plan Risk Management Inputs,also project management plan,areas including,affected scope,also project management plan areas including a...
1,Project charter,The project charter can provide various input...,Plan Risk Management,Plan Risk Management Inputs,project charter,provide various,high level risks,project charter provide various high level risks
2,Stakeholder register,"The stakeholder register, which contains all ...",Plan Risk Management,Plan Risk Management Inputs,which,provides,roles,which provides roles
3,Enterprise Environmental Factors,The enterprise environmental factors that can...,Plan Risk Management,Plan Risk Management Inputs,organization,include,risk,organization include risk
4,organizational Process Assets,The organizational process assets that can in...,Plan Risk Management,Plan Risk Management Inputs,• decision Lessons,learned,• decision making,• decision Lessons learned • decision making
...,...,...,...,...,...,...,...,...
69,Work Performance Information,Work Performance Information. Work performanc...,Control Risks,Control Risks Outputs,Work performance information,making,Control Risks mechanism,Work performance information making Control Ri...
70,change requests,change requests. Implementing contingency pla...,Control Risks,Control Risks Outputs,future performance,are,project management plan,future performance are project management plan
71,Project Management Plan updates,Project Management Plan updates. If the appro...,Control Risks,Control Risks Outputs,2013 Sanchez PMI copy,MANAGEMENT,PMI Member distribution,2013 Sanchez PMI copy MANAGEMENT PMI Member di...
72,Project documents updates,Project documents updates. Project documents ...,Control Risks,Control Risks Outputs,managers,plan for,future projects,managers plan for future projects


# Building our Ontology

## Initializing our Graph

In [48]:
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer
g = Graph()
g.bind("owl",OWL)
g.bind("pr","http://example.org/project#")
ns_url = "http://example.org/project#"
g.add((URIRef('http://example.org/project#'),RDF.type, OWL.Ontology))

<Graph identifier=N3c8ba6f105ed47e9927def6400ad1656 (<class 'rdflib.graph.Graph'>)>

## Add classes and sub-classes

In [49]:
for c in df_concept['annotation']:
    cl = URIRef(ns_url+c.replace(" ","_"))
    g.add((cl,RDF.type, OWL.Class))
    for i in range (len(df_concept)):
        if df_concept.loc[i,'annotation'] == c :
            clp = URIRef(ns_url +df_concept.loc[i,'concept'].replace(" ","_"))
            g.add((cl, RDFS.subClassOf , clp))

http://example.org/project#_data_Gathering_and_representation_techniques._•	__Interviewing._Interviewing_techniques_draw_on_experience_and_historical_data_to_quantify_the_probability_._and_impact_of_risks_on_project_objectives._The_information_needed_depends_upon_the_type_of_probability_._distributions_that_will_be_used._For_instance,_information_would_be_gathered_on_the_optimistic_(low),_._pessimistic_(high),_and_most_likely_scenarios_for_some_commonly_used_distributions._Examples_of_three-._point_estimates_for_cost_are_shown_in_Figure_11-13._Additional_information_on_three-point_estimates_._appears_in_Estimate_Activity_Durations_(Section_6.5)_and_Estimate_Costs_(Section_7.2)._Documenting_the_._rationale_of_the_risk_ranges_and_the_assumptions_behind_them_are_important_components_of_the_risk_._interview_because_they_can_provide_insight_on_the_reliability_and_credibility_of_the_analysis.._Range_of_Project_Cost_Estimates_._WBS_Element_Low_Most_Likely_High._Design_$4M_$6M_$10M._Build_$16M

## Add Data-properties

In [50]:
for i in range(len(df_concept)): 
    c = URIRef(ns_url+df_concept.loc[i,'individuals' ] .replace(" ", "_") )
    domaine = URIRef(ns_url+df_concept.loc[i,'process'].replace(" ","_"))
    g.add((c,RDF.type,OWL.DatatypeProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,XSD.string))

## Add Object-properties


In [51]:
for i in range(len(df_concept)):
    c = URIRef(ns_url + df_concept.loc[i, 'pattern'].replace(" ", "_"))
    domaine = URIRef(ns_url + df_concept.loc[i, 'process'].replace(" ", "_"))
    rang = URIRef(ns_url + df_concept.loc[i, 'object'].replace(" ", "_"))
    g.add((c, RDF.type, OWL.ObjectProperty))


## Add Individuals

In [52]:
for i in range(len(df_concept)): 
    c = URIRef(ns_url+df_concept.loc[i,'concept'].replace(" ","_")) 
    ind = URIRef(ns_url+df_concept.loc[i,'individuals'].replace(" ","_")) 
    g.add((ind,RDF.type,c)) 
    

In [53]:
for i in range(len(df_concept)): 
    c = URIRef(ns_url+df_concept.loc[i,'concept'].replace(" ","_")) 
    ind = URIRef(ns_url+df_concept.loc[i,'individuals'].replace(" ","_")) 
    g.add((ind,RDF.type,OWL.NamedIndividual)) 


In [54]:
for i in range(len(df_concept)): 
    ind = URIRef(ns_url+df_concept.loc[i,'subject'].replace(" ","_")) 
    obj = URIRef(ns_url+df_concept.loc[i,'object'].replace(" ","_")) 
    objprop = URIRef(ns_url+df_concept.loc[i,'individuals'].replace(" ","_")) 
    g.add((ind,objprop,obj)) 


## Now we save our file 

In [55]:
with open('PMBOK1.owl', 'w', encoding='utf-8') as output:
    output.write(g.serialize(format="pretty-xml"))